In [ ]:
import rasterio

from rasterio.plot import show
from palettable.colorbrewer.diverging import RdBu_5 as RedBlueCmap

RED_BLUE_CMAP = RedBlueCmap.mpl_colormap
from common import *

## Snow Depth comparison 

In [ ]:
differences = {}
dates = ['0331', '0524']
FILE_PREFIX = 'depth_difference_50m_2018'
home = PurePath(f'{Path.home()}/scratch/iSnobal/2018-ASO-data/Depth/')

In [ ]:
for date in dates:
    snow_nc = f'{FILE_PREFIX}{date}.tif'
    file = rasterio.open(home.joinpath(snow_nc).as_posix())
    differences[date] = file.read(1, masked=True).compressed()

In [ ]:
def violin_plot(ax, data, title, color):
    vp = ax.violinplot(
        data,
        showmeans=False, showextrema=False, showmedians=True,
    )
    ax.set_xticks([])
    ax.axhline(0, c='black', ls=':')
    
    for pc in vp['bodies']:
        pc.set_facecolor(color)
        pc.set_edgecolor('black')
    
    vp['cmedians'].set_color('black')

    return vp

In [ ]:
def area_plot(date):
    snow_nc = f'{FILE_PREFIX}{date}.tif'
    file = rasterio.open(home.joinpath(snow_nc).as_posix())
    data = file.read(1, masked=True)

    fig, (ax1, ax2) = plt.subplots(
        1, 2, figsize=(6.3,4), 
        gridspec_kw={'width_ratios': [3, 7]},
        dpi=300, 
    )
    fig.set_facecolor('lightgrey')
    ax2.set_facecolor('lightslategrey')

    plt_data = ax2.imshow(
        data,
        vmin=-2,
        vmax=2,
        cmap=RED_BLUE_CMAP
    )
    show(
        data,
        ax=ax2,
        transform=file.transform, 
        vmin=-2,
        vmax=2,
        cmap=RED_BLUE_CMAP
    )
    ax2.tick_params(axis='both', direction='inout', length=7, labelsize=9, pad=0)
    ax2.tick_params(axis='y', labelrotation=45)
    ax2.ticklabel_format(axis='both', style='plain')
    ax2.xaxis.set_ticks_position('both')
    ax2.yaxis.set_ticks_position('both')
    
    fig.colorbar(plt_data, shrink=0.9, extend='both', label='Depth Difference (m)')
    fig.suptitle(datetime.datetime.strptime(f"2018{date}", '%Y%m%d').date().isoformat(), y=0.94)
        
    violin_plot(ax1, differences[date], date, 'orange')
    ax1.set_ylabel('Snow Depth (m)')
    ax1.set_ylim([-2, 2])
    ax1.tick_params(axis='y', labelsize=9)
    plt.savefig(f"/data/iSnobal/figures/ASO_depth_difference_{date}.png")


In [ ]:
for date in dates:
    area_plot(date)
    print(f'Median - {}: {np.median(differences[date]):.2f}')
